In [1]:
import cv2
import numpy as np
import pytesseract
from transformers import AutoTokenizer , AutoModelForSequenceClassification
from scipy.special import softmax
import torch
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix, accuracy_score

c:\Users\dhruvil\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
# Load image
img_path = 'tweet05.jpg'
img = cv2.imread(img_path)

In [37]:
if img is None:
    print(f'Failed to load image from file: {img_path}')
else:
    # Pre-processing
    def preprocess_image(img):
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Threshold the image
        _, thresh = cv2.threshold(gray, 210, 230, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Dilate the image
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
        dilated = cv2.dilate(thresh, kernel, iterations=2)
        
        # Erode the image
        eroded = cv2.erode(dilated, kernel, iterations=2)
        
        #  Gaussian blur
        blurred = cv2.GaussianBlur(eroded, (1, 1), 0)
        
        return blurred

In [38]:
if not isinstance(img, np.ndarray):
        print(f'img is not a valid numpy array: {img}')
else:
        # Apply pre-processing to image
        preprocessed_img = preprocess_image(img)

        # Run OCR on pre-processed image
        ocr_text = pytesseract.image_to_string(preprocessed_img)
        print(ocr_text)
    

Virat Kohli @
@imVkohli

Numero Uno showing why he's the best in the world.

Didn't watch it live but I'm sure this was another video
game innings by him. 4 @surya_14kumar

2:35 PM - Nov 20, 2022 - Twitter for Android



In [39]:
tweet = ocr_text + '😊'

In [40]:
tweet_words = []
for word in tweet.split():
    word = word.strip()  # Remove leading/trailing whitespace
    if word.startswith('@') and len(word) > 1:
        word = '@user'
    elif word.startswith('http'):
        word = "http"
    tweet_words.append(word)

# Create a new list with '\n' removed from each word
tweet_words_no_newline = [word.replace('\n', '') for word in tweet_words]
revised_tweet = " ".join(tweet_words_no_newline)
print(revised_tweet)

Virat Kohli @ @user Numero Uno showing why he's the best in the world. Didn't watch it live but I'm sure this was another video game innings by him. 4 @user 2:35 PM - Nov 20, 2022 - Twitter for Android 😊


In [41]:
# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

In [42]:
# sentiment analysis
encoded_tweet = tokenizer(revised_tweet, return_tensors='pt')
# output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
output = model(**encoded_tweet)

scores = output[0][0].detach().numpy()
scores = softmax(scores)

for i in range(len(scores)):
    
    l = labels[i]
    s = scores[i]
    print(l,s)

Negative 0.0044868956
Neutral 0.08884943
Positive 0.90666366


In [43]:

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Preprocessing function
def preprocess_text(text):
    # Remove punctuation and convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    # Join tokens back into text
    text = ' '.join(tokens)
    return text

# Preprocess text
preprocessed_text = preprocess_text(revised_tweet)

# Tokenize text and convert to input IDs and attention mask
inputs = tokenizer(preprocessed_text, padding=True, truncation=True, return_tensors='pt')
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# Make prediction
with torch.no_grad():
    outputs = model(input_ids, attention_mask)
    scores = outputs[0]
    probs = softmax(scores, axis=1).tolist()[0]

# Print results
labels = ['Negative', 'Neutral', 'Positive']
for label, prob in zip(labels, probs):
    print(f'{label}: {prob}')


Negative: 0.607390284538269
Neutral: 0.11966809630393982
Positive: 0.09580015391111374
